In [142]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime, timedelta
import yfinance as yf

In [158]:
# Total movement 9/9 - 9/20
def get_stock_movement(ticker, start_date, end_date):
    try:
        # Adjust the end_date to include the intended day
        end_date_adjusted = (datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

        stock_data = yf.download(ticker, start=start_date, end=end_date_adjusted, progress=False)

        if stock_data.empty:
            return {"error": f"No data found for {ticker} between {start_date} and {end_date}"}

        opening_price = stock_data['Open'].iloc[0]
        closing_price = stock_data['Close'].iloc[-1]
        peak_price = stock_data['High'].max()

        percentage_total = ((closing_price - opening_price) / opening_price) * 100

        return {
            "ticker": ticker,
            "9/9-9/20_total": round(percentage_total, 2)  # Rounded to two decimal places
        }

    except Exception as e:
        return {"error": str(e)}

# List of tickers
tickers = ['GP', 'CVR', 'JCTC', 'BRN', 'FCUV', 'SIF', 'DAIO', 'SVT', 'OESX', 'AP',
           'OPTT', 'MESA', 'GLBS', 'FLUX', 'GTEC', 'FTEK', 'APWC', 'EDRY', 'SYPR', 'BWEN',
           'BEEM', 'CVU', 'ELSE', 'TOMZ', 'UGRO', 'OP', 'ASTC', 'PETZ', 'EKSO', 'AQMS',
           'YHGJ', 'AIRI', 'PRZO', 'OCC', 'LIQT', 'HLP', 'CCTG', 'RETO', 'DPRO', 'UAVS', 'GDC', 'KNW', 
           'BNGO', 'NISN', 'HUDI', 'DTCK', 'ZKIN', 'PMEC', 'HYFM', 'TPCS', 'FEAM', 'AXDX', 'VGAS', 'MTEN',
            'DSWL', 'GLST', 'MIND', 'FSI', 'IPWR', 'ESGL', 'CPTN', 'CLIR', 'JUNE', 'APT', 'TPIC', 'GWH', 'PPSI']
           

# Start and end dates
start_date = "2024-09-09"
end_date = "2024-09-20"

# Initialize empty list to hold the data
data = []

# Loop through tickers
for ticker in tickers:
    result = get_stock_movement(ticker, start_date, end_date)
    if "error" in result:
        print(f"Error for {ticker}: {result['error']}")
    else:
        data.append(result)

# Create DataFrame
gp_comp_df = pd.DataFrame(data)

# Display the DataFrame with the specified columns
gp_comp_df = gp_comp_df[['ticker', '9/9-9/20_total']].astype({'ticker': str, '9/9-9/20_total': float})

print(gp_comp_df)


   ticker  9/9-9/20_total
0      GP           43.19
1     CVR           12.10
2    JCTC           15.15
3     BRN            6.48
4    FCUV           -0.53
..    ...             ...
62   JUNE            1.75
63    APT            2.76
64   TPIC           23.12
65    GWH           -1.92
66   PPSI           10.65

[67 rows x 2 columns]


C:\Users\19413\AppData\Local\Temp\ipykernel_9604\3778959935.py:54: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  gp_comp_df = gp_comp_df[['ticker', '9/9-9/20_total']].astype({'ticker': str, '9/9-9/20_total': float})


In [159]:
gp_total= gp_comp_df['9/9-9/20_total']

stat, p_value = stats.shapiro(gp_total)

if p_value > 0.05:
    print("Data appears normally distributed (fail to reject H0)")
else:
    print("Data does not appear normally distributed (reject H0)")
    

Data does not appear normally distributed (reject H0)


In [160]:
#For total price movement 9/9-9/20
from scipy.stats import mannwhitneyu

just_gp_total = gp_comp_df.loc[gp_comp_df['ticker'] == 'GP', '9/9-9/20_total'].values[0]

# Extract the values for the rest of the stocks
rest_totals = gp_comp_df.loc[gp_comp_df['ticker'] != 'GP', '9/9-9/20_total']

u_stat, p_value = stats.mannwhitneyu(just_gp_total, rest_totals, alternative = 'two-sided')
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')


Mann Whitney U Statistics: 64.0
p value: 0.0896


In [161]:
import numpy as np
from scipy import stats
##Perplexity version
def hodges_lehmann_interval(just_gp_total, rest_totals, alpha=0.05):
    if np.isscalar(just_gp_total):
        just_gp_total = [just_gp_total]
    if np.isscalar(rest_totals):
        rest_totals = [rest_totals]

    pairwise_differences = [x1 - x2 for x1 in just_gp_total for x2 in rest_totals]
    pairwise_differences.sort()

    hl_estimator = np.median(pairwise_differences)

    n1, n2 = len(just_gp_total), len(rest_totals)
    n = n1 * n2

    # Calculate the critical value
    z = stats.norm.ppf(1 - alpha/2)

    # Calculate the standard error
    se = np.sqrt((n1 + n2 + 1) / (12 * n1 * n2))

    # Calculate the confidence interval
    margin = z * se * np.sqrt(n)
    lower_rank = int(np.ceil((n - margin) / 2) - 1)
    upper_rank = int(np.floor((n + margin) / 2) - 1)

    lower_bound = pairwise_differences[max(0, lower_rank)]
    upper_bound = pairwise_differences[min(n - 1, upper_rank)]

    return {
        "Hodges-Lehmann Estimator": hl_estimator,
        "Confidence Interval": (lower_bound, upper_bound)
    }

# The rest of your code remains the same
u_stat, p_value = stats.mannwhitneyu(just_gp_total, rest_totals, alternative='two-sided')

result = hodges_lehmann_interval(just_gp_total, rest_totals, alpha=0.05)
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')
print(f"Hodges-Lehmann Estimator: {result['Hodges-Lehmann Estimator']}")
print(f"Confidence Interval: {result['Confidence Interval']}")


Mann Whitney U Statistics: 64.0
p value: 0.0896
Hodges-Lehmann Estimator: 40.59
Confidence Interval: (np.float64(40.169999999999995), np.float64(40.839999999999996))


In [162]:
#Peak movement 9/9-9/20
def get_stock_movement(ticker, start_date, end_date):
    try:
        # Adjust the end_date to include the intended day
        end_date_adjusted = (datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

        stock_data = yf.download(ticker, start=start_date, end=end_date_adjusted, progress=False)

        if stock_data.empty:
            return {"error": f"No data found for {ticker} between {start_date} and {end_date}"}

        opening_price = stock_data['Open'].iloc[0]
        peak_price = stock_data['High'].max()

        percentage_peak = ((peak_price - opening_price) / opening_price) * 100

        return {
            "ticker": ticker,
            "9/9-9/16_peak": round(percentage_peak, 2)  # Rounded to two decimal places
        }

    except Exception as e:
        return {"error": str(e)}

# List of tickers
tickers = ['GP', 'CVR', 'JCTC', 'BRN', 'FCUV', 'SIF', 'DAIO', 'SVT', 'OESX', 'AP',
           'OPTT', 'MESA', 'GLBS', 'FLUX', 'GTEC', 'FTEK', 'APWC', 'EDRY', 'SYPR', 'BWEN',
           'BEEM', 'CVU', 'ELSE', 'TOMZ', 'UGRO', 'OP', 'ASTC', 'PETZ', 'EKSO', 'AQMS',
           'YHGJ', 'AIRI', 'PRZO', 'OCC', 'LIQT', 'HLP', 'CCTG', 'RETO', 'DPRO', 'UAVS', 'GDC', 'KNW', 
           'BNGO', 'NISN', 'HUDI', 'DTCK', 'ZKIN', 'PMEC', 'HYFM', 'TPCS', 'FEAM', 'AXDX', 'VGAS', 'MTEN',
            'DSWL', 'GLST', 'MIND', 'FSI', 'IPWR', 'ESGL', 'CPTN', 'CLIR', 'JUNE', 'APT', 'TPIC', 'GWH', 'PPSI']
           

# Start and end dates
start_date = "2024-09-09"
end_date = "2024-09-16"

# Initialize empty list to hold the data
data = []

# Loop through tickers
for ticker in tickers:
    result = get_stock_movement(ticker, start_date, end_date)
    if "error" in result:
        print(f"Error for {ticker}: {result['error']}")
    else:
        data.append(result)

# Create DataFrame
gp_comp_df = pd.DataFrame(data)

# Display the DataFrame with the specified columns
gp_comp_df = gp_comp_df[['ticker', '9/9-9/16_peak']].astype({'ticker': str, '9/9-9/16_peak': float})

print(gp_comp_df)


   ticker  9/9-9/16_peak
0      GP         121.19
1     CVR           2.40
2    JCTC          11.19
3     BRN           5.09
4    FCUV          44.47
..    ...            ...
62   JUNE          11.64
63    APT           0.00
64   TPIC          24.74
65    GWH           7.49
66   PPSI          18.60

[67 rows x 2 columns]


C:\Users\19413\AppData\Local\Temp\ipykernel_9604\184896813.py:53: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  gp_comp_df = gp_comp_df[['ticker', '9/9-9/16_peak']].astype({'ticker': str, '9/9-9/16_peak': float})


In [163]:
#For peak movement 9/9-9/16

just_gp_peak = gp_comp_df.loc[0, '9/9-9/16_peak']
rest_peak = gp_comp_df['9/9-9/16_peak'][1:]

u_stat, p_value = stats.mannwhitneyu(just_gp_peak, rest_peak, alternative = 'two-sided')
print(f'Mann Whitney U Statistic: {u_stat}')
print(f'p value: {p_value:.4f}')

Mann Whitney U Statistic: 66.0
p value: 0.0928


In [150]:
def hodges_lehmann_interval(just_gp_peak, rest_peak, alpha=0.05):
    if np.isscalar(just_gp_peak):
        just_gp_peak = [just_gp_peak]
    if np.isscalar(rest_peak):
        rest_peak = [rest_peak]

    pairwise_differences = [x1 - x2 for x1 in just_gp_peak for x2 in rest_peak]
    pairwise_differences.sort()

    hl_estimator = np.median(pairwise_differences)

    n = len(pairwise_differences)

    _, u_stat = mannwhitneyu(just_gp_peak, rest_peak, alternative='two-sided')

    lower_rank = int(max(0, u_stat - 1.96 * np.sqrt(n)))  # Ensure lower rank is >= 0
    upper_rank = int(min(n - 1, u_stat + 1.96 * np.sqrt(n)))  # Ensure upper rank < n

    lower_bound = pairwise_differences[lower_rank]
    upper_bound = pairwise_differences[upper_rank]

    return {
        "Hodges-Lehmann Estimator": hl_estimator,
        "Confidence Interval": (lower_bound, upper_bound)
    }

u_stat, p_value = stats.mannwhitneyu(just_gp_peak, rest_peak, alternative = 'two-sided')


result = hodges_lehmann_interval(just_gp_peak, rest_peak, alpha=0.05)
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')
print(f"Hodges-Lehmann Estimator: {result['Hodges-Lehmann Estimator']}")
print(f"Confidence Interval: {result['Confidence Interval']}")

Mann Whitney U Statistics: 66.0
p value: 0.0928
Hodges-Lehmann Estimator: 112.25999999999999
Confidence Interval: (np.float64(56.849999999999994), np.float64(106.19))


In [164]:
import numpy as np
from scipy import stats
##Perplexity version
def hodges_lehmann_interval(just_gp_peak, rest_peak, alpha=0.05):
    if np.isscalar(just_gp_peak):
        just_gp_peak = [just_gp_peak]
    if np.isscalar(rest_peak):
        rest_peak = [rest_peak]

    pairwise_differences = [x1 - x2 for x1 in just_gp_peak for x2 in rest_peak]
    pairwise_differences.sort()

    hl_estimator = np.median(pairwise_differences)

    n1, n2 = len(just_gp_peak), len(rest_peak)
    n = n1 * n2

    # Calculate the critical value
    z = stats.norm.ppf(1 - alpha/2)

    # Calculate the standard error
    se = np.sqrt((n1 + n2 + 1) / (12 * n1 * n2))

    # Calculate the confidence interval
    margin = z * se * np.sqrt(n)
    lower_rank = int(np.ceil((n - margin) / 2) - 1)
    upper_rank = int(np.floor((n + margin) / 2) - 1)

    lower_bound = pairwise_differences[max(0, lower_rank)]
    upper_bound = pairwise_differences[min(n - 1, upper_rank)]

    return {
        "Hodges-Lehmann Estimator": hl_estimator,
        "Confidence Interval": (lower_bound, upper_bound)
    }

# The rest of your code remains the same
u_stat, p_value = stats.mannwhitneyu(just_gp_peak, rest_peak, alternative='two-sided')

result = hodges_lehmann_interval(just_gp_peak, rest_peak, alpha=0.05)
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')
print(f"Hodges-Lehmann Estimator: {result['Hodges-Lehmann Estimator']}")
print(f"Confidence Interval: {result['Confidence Interval']}")


Mann Whitney U Statistics: 66.0
p value: 0.0928
Hodges-Lehmann Estimator: 112.25999999999999
Confidence Interval: (np.float64(111.92999999999999), np.float64(113.28))


In [135]:
#For total movement 10/23-10/30
#(Useless to include because negative)

def get_stock_movement(ticker, start_date, end_date):
    try:
        # Adjust the end_date to include the intended day
        end_date_adjusted = (datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

        stock_data = yf.download(ticker, start=start_date, end=end_date_adjusted, progress=False)

        if stock_data.empty:
            return {"error": f"No data found for {ticker} between {start_date} and {end_date}"}

        opening_price = stock_data['Open'].iloc[0]
        closing_price = stock_data['Close'].iloc[-1]
        peak_price = stock_data['High'].max()

        percentage_total = ((closing_price - opening_price) / opening_price) * 100

        return {
            "ticker": ticker,
            "10/23-10/30_total": round(percentage_total, 2)  # Rounded to two decimal places
        }

    except Exception as e:
        return {"error": str(e)}

# List of tickers
tickers = ['GP', 'CVR', 'JCTC', 'BRN', 'FCUV', 'SIF', 'DAIO', 'SVT', 'OESX', 'AP',
           'OPTT', 'MESA', 'GLBS', 'FLUX', 'GTEC', 'FTEK', 'APWC', 'EDRY', 'SYPR', 'BWEN',
           'BEEM', 'CVU', 'ELSE', 'TOMZ', 'UGRO', 'OP', 'ASTC', 'PETZ', 'EKSO', 'AQMS',
           'YHGJ', 'AIRI', 'PRZO', 'OCC', 'LIQT', 'HLP', 'CCTG', 'RETO', 'DPRO', 'UAVS', 'GDC', 'KNW', 
           'BNGO', 'NISN', 'HUDI', 'DTCK', 'ZKIN', 'PMEC', 'HYFM', 'TPCS', 'FEAM', 'AXDX', 'VGAS', 'MTEN',
            'DSWL', 'GLST', 'MIND', 'FSI', 'IPWR', 'ESGL', 'CPTN', 'CLIR', 'JUNE', 'APT', 'TPIC', 'GWH', 'PPSI']
           

# Start and end dates
start_date = "2024-10-23"
end_date = "2024-10-30"

# Initialize empty list to hold the data
data = []

# Loop through tickers
for ticker in tickers:
    result = get_stock_movement(ticker, start_date, end_date)
    if "error" in result:
        print(f"Error for {ticker}: {result['error']}")
    else:
        data.append(result)

# Create DataFrame
gp_comp_df = pd.DataFrame(data)

# Display the DataFrame with the specified columns
gp_comp_df = gp_comp_df[['ticker', '10/23-10/30_total']].astype({'ticker': str, '10/23-10/30_total': float})



C:\Users\19413\AppData\Local\Temp\ipykernel_9604\1570367238.py:56: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  gp_comp_df = gp_comp_df[['ticker', '10/23-10/30_total']].astype({'ticker': str, '10/23-10/30_total': float})


In [136]:
#For total movement 10/23-10/30
#(Useless to include because negative)

just_gp_total = gp_comp_df.loc[0, '10/23-10/30_total']
rest_totals = gp_comp_df['10/23-10/30_total'][1:]

u_stat, p_value = stats.mannwhitneyu(just_gp_total, rest_totals, alternative = 'two-sided')
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')


Mann Whitney U Statistics: 2.0
p value: 0.1148


In [165]:
#Peak movement 10/23 - 10/25
def get_stock_movement(ticker, start_date, end_date):
    try:
        # Adjust the end_date to include the intended day
        end_date_adjusted = (datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

        stock_data = yf.download(ticker, start=start_date, end=end_date_adjusted, progress=False)

        if stock_data.empty:
            return {"error": f"No data found for {ticker} between {start_date} and {end_date}"}

        opening_price = stock_data['Open'].iloc[0]
        peak_price = stock_data['High'].max()

        percentage_peak = ((peak_price - opening_price) / opening_price) * 100

        return {
            "ticker": ticker,
            "10/23-10/25_peak": round(percentage_peak, 2)  # Rounded to two decimal places
        }

    except Exception as e:
        return {"error": str(e)}

# List of tickers
tickers = ['GP', 'CVR', 'JCTC', 'BRN', 'FCUV', 'SIF', 'DAIO', 'SVT', 'OESX', 'AP',
           'OPTT', 'MESA', 'GLBS', 'FLUX', 'GTEC', 'FTEK', 'APWC', 'EDRY', 'SYPR', 'BWEN',
           'BEEM', 'CVU', 'ELSE', 'TOMZ', 'UGRO', 'OP', 'ASTC', 'PETZ', 'EKSO', 'AQMS',
           'YHGJ', 'AIRI', 'PRZO', 'OCC', 'LIQT', 'HLP', 'CCTG', 'RETO', 'DPRO', 'UAVS', 'GDC', 'KNW', 
           'BNGO', 'NISN', 'HUDI', 'DTCK', 'ZKIN', 'PMEC', 'HYFM', 'TPCS', 'FEAM', 'AXDX', 'VGAS', 'MTEN',
            'DSWL', 'GLST', 'MIND', 'FSI', 'IPWR', 'ESGL', 'CPTN', 'CLIR', 'JUNE', 'APT', 'TPIC', 'GWH', 'PPSI']
           

# Start and end dates
start_date = "2024-10-23"
end_date = "2024-10-25"

# Initialize empty list to hold the data
data = []

# Loop through tickers
for ticker in tickers:
    result = get_stock_movement(ticker, start_date, end_date)
    if "error" in result:
        print(f"Error for {ticker}: {result['error']}")
    else:
        data.append(result)

# Create DataFrame
gp_comp_df = pd.DataFrame(data)

# Display the DataFrame with the specified columns
gp_comp_df = gp_comp_df[['ticker', '10/23-10/25_peak']].astype({'ticker': str, '10/23-10/25_peak': float})

print(gp_comp_df)

   ticker  10/23-10/25_peak
0      GP             49.19
1     CVR              2.72
2    JCTC              5.06
3     BRN              1.48
4    FCUV              3.35
..    ...               ...
62   JUNE              9.32
63    APT              1.35
64   TPIC              2.27
65    GWH              4.57
66   PPSI              3.06

[67 rows x 2 columns]


C:\Users\19413\AppData\Local\Temp\ipykernel_9604\515122368.py:53: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  gp_comp_df = gp_comp_df[['ticker', '10/23-10/25_peak']].astype({'ticker': str, '10/23-10/25_peak': float})


In [166]:
#For peak movement 10/23-10/25

just_gp_peak = gp_comp_df.loc[0, '10/23-10/25_peak']
rest_peak = gp_comp_df['10/23-10/25_peak'][1:]

u_stat, p_value = stats.mannwhitneyu(just_gp_peak, rest_peak, alternative = 'two-sided')
print(f'Mann Whitney U Statistic: {u_stat}')
print(f'p value: {p_value:.4f}')

Mann Whitney U Statistic: 64.0
p value: 0.1143


In [153]:
def hodges_lehmann_interval(just_gp_peak, rest_peak, alpha=0.05):
    if np.isscalar(just_gp_peak):
        just_gp_peak = [just_gp_peak]
    if np.isscalar(rest_peak):
        rest_peak = [rest_peak]

    pairwise_differences = [x1 - x2 for x1 in just_gp_peak for x2 in rest_peak]
    pairwise_differences.sort()

    hl_estimator = np.median(pairwise_differences)

    n = len(pairwise_differences)

    _, u_stat = mannwhitneyu(just_gp_peak, rest_peak, alternative='two-sided')

    lower_rank = int(max(0, u_stat - 1.96 * np.sqrt(n)))  # Ensure lower rank is >= 0
    upper_rank = int(min(n - 1, u_stat + 1.96 * np.sqrt(n)))  # Ensure upper rank < n

    lower_bound = pairwise_differences[lower_rank]
    upper_bound = pairwise_differences[upper_rank]

    return {
        "Hodges-Lehmann Estimator": hl_estimator,
        "Confidence Interval": (lower_bound, upper_bound)
    }

u_stat, p_value = stats.mannwhitneyu(just_gp_peak, rest_peak, alternative = 'two-sided')


result = hodges_lehmann_interval(just_gp_peak, rest_peak, alpha=0.05)
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')
print(f"Hodges-Lehmann Estimator: {result['Hodges-Lehmann Estimator']}")
print(f"Confidence Interval: {result['Confidence Interval']}")

Mann Whitney U Statistics: 64.0
p value: 0.1143
Hodges-Lehmann Estimator: 46.385
Confidence Interval: (np.float64(-80.28999999999999), np.float64(44.17))


In [167]:
import numpy as np
from scipy import stats
##Perplexity version
def hodges_lehmann_interval(just_gp_peak, rest_peak, alpha=0.05):
    if np.isscalar(just_gp_peak):
        just_gp_peak = [just_gp_peak]
    if np.isscalar(rest_peak):
        rest_peak = [rest_peak]

    pairwise_differences = [x1 - x2 for x1 in just_gp_peak for x2 in rest_peak]
    pairwise_differences.sort()

    hl_estimator = np.median(pairwise_differences)

    n1, n2 = len(just_gp_peak), len(rest_peak)
    n = n1 * n2

    # Calculate the critical value
    z = stats.norm.ppf(1 - alpha/2)

    # Calculate the standard error
    se = np.sqrt((n1 + n2 + 1) / (12 * n1 * n2))

    # Calculate the confidence interval
    margin = z * se * np.sqrt(n)
    lower_rank = int(np.ceil((n - margin) / 2) - 1)
    upper_rank = int(np.floor((n + margin) / 2) - 1)

    lower_bound = pairwise_differences[max(0, lower_rank)]
    upper_bound = pairwise_differences[min(n - 1, upper_rank)]

    return {
        "Hodges-Lehmann Estimator": hl_estimator,
        "Confidence Interval": (lower_bound, upper_bound)
    }

# The rest of your code remains the same
u_stat, p_value = stats.mannwhitneyu(just_gp_peak, rest_peak, alternative='two-sided')

result = hodges_lehmann_interval(just_gp_peak, rest_peak, alpha=0.05)
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')
print(f"Hodges-Lehmann Estimator: {result['Hodges-Lehmann Estimator']}")
print(f"Confidence Interval: {result['Confidence Interval']}")


Mann Whitney U Statistics: 64.0
p value: 0.1143
Hodges-Lehmann Estimator: 46.385
Confidence Interval: (np.float64(46.129999999999995), np.float64(46.47))


In [168]:
#For peak movement 11/26-11/27

def get_stock_movement(ticker, start_date, end_date):
    try:
        # Adjust the end_date to include the intended day
        end_date_adjusted = (datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

        stock_data = yf.download(ticker, start=start_date, end=end_date_adjusted, progress=False)

        if stock_data.empty:
            return {"error": f"No data found for {ticker} between {start_date} and {end_date}"}

        opening_price = stock_data['Open'].iloc[0]
        closing_price = stock_data['Close'].iloc[-1]
        peak_price = stock_data['High'].max()

        percentage_total = ((closing_price - opening_price) / opening_price) * 100

        return {
            "ticker": ticker,
            "11/26-11/27_peak": round(percentage_total, 2)  # Rounded to two decimal places
        }

    except Exception as e:
        return {"error": str(e)}

# List of tickers
tickers = ['GP', 'CVR', 'JCTC', 'BRN', 'FCUV', 'SIF', 'DAIO', 'SVT', 'OESX', 'AP',
           'OPTT', 'MESA', 'GLBS', 'FLUX', 'GTEC', 'FTEK', 'APWC', 'EDRY', 'SYPR', 'BWEN',
           'BEEM', 'CVU', 'ELSE', 'TOMZ', 'UGRO', 'OP', 'ASTC', 'PETZ', 'EKSO', 'AQMS',
           'YHGJ', 'AIRI', 'PRZO', 'OCC', 'LIQT', 'HLP', 'CCTG', 'RETO', 'DPRO', 'UAVS', 'GDC', 'KNW', 
           'BNGO', 'NISN', 'HUDI', 'DTCK', 'ZKIN', 'PMEC', 'HYFM', 'TPCS', 'FEAM', 'AXDX', 'VGAS', 'MTEN',
            'DSWL', 'GLST', 'MIND', 'FSI', 'IPWR', 'ESGL', 'CPTN', 'CLIR', 'JUNE', 'APT', 'TPIC', 'GWH', 'PPSI']
           

# Start and end dates
start_date = "2024-11-26"
end_date = "2024-11-27"

# Initialize empty list to hold the data
data = []

# Loop through tickers
for ticker in tickers:
    result = get_stock_movement(ticker, start_date, end_date)
    if "error" in result:
        print(f"Error for {ticker}: {result['error']}")
    else:
        data.append(result)

# Create DataFrame
gp_comp_df = pd.DataFrame(data)

# Display the DataFrame with the specified columns
gp_comp_df = gp_comp_df[['ticker', '11/26-11/27_peak']].astype({'ticker': str, '11/26-11/27_peak': float})

print(gp_comp_df)


   ticker  11/26-11/27_peak
0      GP             10.99
1     CVR             -2.67
2    JCTC              5.41
3     BRN              4.12
4    FCUV              4.10
..    ...               ...
62   JUNE              0.43
63    APT             -0.56
64   TPIC             -9.50
65    GWH              3.22
66   PPSI             -0.50

[67 rows x 2 columns]


C:\Users\19413\AppData\Local\Temp\ipykernel_9604\646061105.py:55: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  gp_comp_df = gp_comp_df[['ticker', '11/26-11/27_peak']].astype({'ticker': str, '11/26-11/27_peak': float})


In [169]:
#For peak movement 11/23-11/25

just_gp_peak = gp_comp_df.loc[0, '11/26-11/27_peak']
rest_peak = gp_comp_df['11/26-11/27_peak'][1:]

u_stat, p_value = stats.mannwhitneyu(just_gp_peak, rest_peak, alternative = 'two-sided')
print(f'Mann Whitney U Statistic: {u_stat}')
print(f'p value: {p_value:.4f}')

Mann Whitney U Statistic: 62.0
p value: 0.1405


In [156]:
def hodges_lehmann_interval(just_gp_peak, rest_peak, alpha=0.05):
    if np.isscalar(just_gp_peak):
        just_gp_peak = [just_gp_peak]
    if np.isscalar(rest_peak):
        rest_peak = [rest_peak]

    pairwise_differences = [x1 - x2 for x1 in just_gp_peak for x2 in rest_peak]
    pairwise_differences.sort()

    hl_estimator = np.median(pairwise_differences)

    n = len(pairwise_differences)

    _, u_stat = mannwhitneyu(just_gp_peak, rest_peak, alternative='two-sided')

    lower_rank = int(max(0, u_stat - 1.96 * np.sqrt(n)))  # Ensure lower rank is >= 0
    upper_rank = int(min(n - 1, u_stat + 1.96 * np.sqrt(n)))  # Ensure upper rank < n

    lower_bound = pairwise_differences[lower_rank]
    upper_bound = pairwise_differences[upper_rank]

    return {
        "Hodges-Lehmann Estimator": hl_estimator,
        "Confidence Interval": (lower_bound, upper_bound)
    }

u_stat, p_value = stats.mannwhitneyu(just_gp_peak, rest_peak, alternative = 'two-sided')


result = hodges_lehmann_interval(just_gp_peak, rest_peak, alpha=0.05)
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')
print(f"Hodges-Lehmann Estimator: {result['Hodges-Lehmann Estimator']}")
print(f"Confidence Interval: {result['Confidence Interval']}")

Mann Whitney U Statistics: 62.0
p value: 0.1405
Hodges-Lehmann Estimator: 10.99
Confidence Interval: (np.float64(-74.01), np.float64(7.94))


In [170]:
import numpy as np
from scipy import stats
##Perplexity version
def hodges_lehmann_interval(just_gp_peak, rest_peak, alpha=0.05):
    if np.isscalar(just_gp_peak):
        just_gp_peak = [just_gp_peak]
    if np.isscalar(rest_peak):
        rest_peak = [rest_peak]

    pairwise_differences = [x1 - x2 for x1 in just_gp_peak for x2 in rest_peak]
    pairwise_differences.sort()

    hl_estimator = np.median(pairwise_differences)

    n1, n2 = len(just_gp_peak), len(rest_peak)
    n = n1 * n2

    # Calculate the critical value
    z = stats.norm.ppf(1 - alpha/2)

    # Calculate the standard error
    se = np.sqrt((n1 + n2 + 1) / (12 * n1 * n2))

    # Calculate the confidence interval
    margin = z * se * np.sqrt(n)
    lower_rank = int(np.ceil((n - margin) / 2) - 1)
    upper_rank = int(np.floor((n + margin) / 2) - 1)

    lower_bound = pairwise_differences[max(0, lower_rank)]
    upper_bound = pairwise_differences[min(n - 1, upper_rank)]

    return {
        "Hodges-Lehmann Estimator": hl_estimator,
        "Confidence Interval": (lower_bound, upper_bound)
    }

# The rest of your code remains the same
u_stat, p_value = stats.mannwhitneyu(just_gp_peak, rest_peak, alternative='two-sided')

result = hodges_lehmann_interval(just_gp_peak, rest_peak, alpha=0.05)
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')
print(f"Hodges-Lehmann Estimator: {result['Hodges-Lehmann Estimator']}")
print(f"Confidence Interval: {result['Confidence Interval']}")


Mann Whitney U Statistics: 62.0
p value: 0.1405
Hodges-Lehmann Estimator: 10.99
Confidence Interval: (np.float64(10.99), np.float64(11.38))


In [171]:
#For combining peak movement p - values
from scipy.stats import chi2

def fisher_combined_pvalue(p_values):
 
    chi2_stat = -2 * np.sum(np.log(p_values))

    df = 2 * len(p_values)

    combined_p_value = 1 - chi2.cdf(chi2_stat, df)

    return combined_p_value

p_values = [0.0928, 0.1143, 0.1405]  # List of individual p-values
combined_p_value = fisher_combined_pvalue(p_values)

print(f"Combined p-value using Fisher's method: {combined_p_value:.4f}")


Combined p-value using Fisher's method: 0.0428
